# Import libs

In [408]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Read data

In [5]:
PATH_TO_DATA = '...//House Prices - Advanced Regression Techniques//Data'

In [6]:
sample_submission = pd.read_csv(f'{PATH_TO_DATA}//sample_submission.csv')

In [8]:
test = pd.read_csv(f'{PATH_TO_DATA}//test.csv')

In [9]:
train = pd.read_csv(f'{PATH_TO_DATA}//train.csv')

# Work place

## Preporation data

In [207]:
train_copy = train.copy()

In [208]:
train_copy.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [209]:
obj_train = train_copy.select_dtypes(include=['object']).copy()

In [210]:
numeric_train = train_copy.drop(obj_train.columns, axis = 1).copy()

In [315]:
def get_obj_numeric_dfs(df: pd.DataFrame):
    obj_train = df.select_dtypes(include=['object']).copy()
    numeric_train = df.drop(obj_train.columns, axis = 1).copy()
    return obj_train, numeric_train

### Preporatino categorical columns

In [319]:
def preporation_categorical_columns(train_copy: pd.DataFrame, obj_train: pd.DataFrame):
    list_columns_nan = obj_train.loc[:, obj_train.isnull().any()].columns
    for col in list_columns_nan:
        val = obj_train[col].value_counts().reset_index()['index'][0]
        obj_train = obj_train.fillna({col: val})
        
    cleanup_nums = {"CentralAir":  {"Y": 1, "N": 0}}
    obj_train = obj_train.replace(cleanup_nums)
    list_columns_obj = obj_train.select_dtypes(include=['object']).columns
    train_copy = train_copy.replace(cleanup_nums)
    train_copy = pd.get_dummies(train_copy, columns=list(list_columns_obj))
    return train_copy

In [211]:
list_columns_nan = obj_train.loc[:, obj_train.isnull().any()].columns
for col in list_columns_nan:
    val = obj_train[col].value_counts().reset_index()['index'][0]
    obj_train = obj_train.fillna({col: val})

In [212]:
cleanup_nums = {"CentralAir":  {"Y": 1, "N": 0}}

In [213]:
obj_train = obj_train.replace(cleanup_nums)

In [214]:
list_columns_obj = obj_train.select_dtypes(include=['object']).columns

In [215]:
train_copy = train_copy.replace(cleanup_nums)

In [216]:
train_copy = pd.get_dummies(train_copy, columns=list(list_columns_obj))

In [217]:
train_copy.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


### Preporation numeric columns

In [322]:
def preporation_numeric_columns(train_copy: pd.DataFrame, numeric_train: pd.DataFrame):
    list_columns_nan = numeric_train.loc[:, numeric_train.isnull().any()].columns
    for col in list_columns_nan:
        val = round(train_copy[col].mean())
        train_copy = train_copy.fillna({col: val})
    return train_copy

In [218]:
numeric_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [219]:
numeric_train.loc[:, numeric_train.isnull().any()].columns

Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')

In [220]:
list_columns_nan = numeric_train.loc[:, numeric_train.isnull().any()].columns
for col in list_columns_nan:
    val = round(train_copy[col].mean())
    train_copy = train_copy.fillna({col: val})

## Calculate predict

### Linear Regressor

In [328]:
y = train_copy['SalePrice']

In [396]:
x = train_copy.drop(['Id','SalePrice'], axis = 1).copy() #[important_features]#

In [409]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=10)

In [410]:
model = LinearRegression()

In [411]:
model.fit(X_train, y_train)

LinearRegression()

In [412]:
#model = LinearRegression().fit(x, y)

In [413]:
r_sq = model.score(X_test, y_test)
print('coefficient of determination:', r_sq)

coefficient of determination: 0.8573783792680627


In [401]:
#print('intercept:', model.intercept_)
#print('slope:', model.coef_)

In [414]:
y_pred_linear = model.predict(X_test)

In [415]:
mean_squared_error(y_test, y_pred_linear, squared=False )

30165.416305221894

### RandomForest Regressor

In [406]:
y = train_copy['SalePrice']

In [407]:
X = train_copy.drop(['Id', 'SalePrice'], axis = 1).copy()

In [660]:
regr_important = RandomForestRegressor(max_depth=17, random_state=10)

In [661]:
regr_important.fit(X_train, y_train)

RandomForestRegressor(max_depth=17, random_state=10)

In [668]:
dict_columns_important = {col: i for col,i in zip(X.columns, regr_important.feature_importances_)}

In [675]:
list_best_20 = sorted(dict_columns_important.items(), key=lambda x: x[1], reverse=True)[:20]

In [678]:
important_features = [i[0] for i in list_best_20]

In [664]:
y_pred_random_for = regr_important.predict(X_test)

In [665]:
mean_squared_error(y_test, y_pred_random_for, squared=False)

28023.60151737956

In [680]:
Xx = X_train[important_features].copy()

In [681]:
regr = RandomForestRegressor(max_depth=17, random_state=10)

In [682]:
regr.fit(Xx, y_train)

RandomForestRegressor(max_depth=17, random_state=10)

In [683]:
y_pred_random_for = regr.predict(X_test[important_features])

In [684]:
mean_squared_error(y_test, y_pred_random_for, squared=False)

28964.937620877405

# Test data

In [316]:
test_obj, test_numeric = get_obj_numeric_dfs(test)

In [317]:
test_copy = test.copy()

In [320]:
test_copy = preporation_categorical_columns(test_copy, test_obj)

In [323]:
test_copy = preporation_numeric_columns(test_copy, test_numeric)

In [349]:
XX = test_copy[important_features]

In [367]:
y_test_pred_linear = model.predict(XX)

In [357]:
y_test_pred_for = regr.predict(XX)

# Compare results

In [379]:
y_compare_test = sample_submission['SalePrice'].values

In [394]:
mean_squared_error(y_compare_test,  y_test_pred_linear, squared=False)

69613.83764364317

In [395]:
mean_squared_error(y_compare_test, y_test_pred_for, squared=False)

70358.93978953353